# Etude des motifs de préférence musicale Lalala 

In [53]:
import pandas as pd 
import numpy as np 

# import des données 
data = pd.read_csv('../Source/responses.csv')

# on remplit les données manquantes avec NaN 
data = data.replace(r'\s+', np.nan)

# remplacer les variables catégorielles 
for column in data.columns : 
    data[column] = data[column].astype('category')
    data[column] = data[column].cat.codes

print(data.shape)

(1010, 150)
Music                         int8
Slow songs or fast songs      int8
Dance                         int8
Folk                          int8
Country                       int8
Classical music               int8
Musical                       int8
Pop                           int8
Rock                          int8
Metal or Hardrock             int8
Punk                          int8
Hiphop, Rap                   int8
Reggae, Ska                   int8
Swing, Jazz                   int8
Rock n roll                   int8
Alternative                   int8
Latino                        int8
Techno, Trance                int8
Opera                         int8
Movies                        int8
Horror                        int8
Thriller                      int8
Comedy                        int8
Romantic                      int8
Sci-fi                        int8
War                           int8
Fantasy/Fairy tales           int8
Animated                      int8
Document

A ce stade, on a des données propres à utiliser. En effet, on a remplacé les variables catégorielles par des variables numériques. On a également remplacé les champs non renseignés par -1, qui est une valeur jamais prise par les variables.

On calcule la matrice de préférence pour l'ensemble du jeu de données. Pour cela, on sépare d'abord les données pour extraire d'une part les champs à comparer (les goûts musicaux) des caractéristiques (données démographiques).

* Music preferences (19 items)
* Movie preferences (12 items)
* Hobbies & interests (32 items)
* Phobias (10 items)
* Health habits (3 items)
* Personality traits, views on life, & opinions (57 items)
* Spending habits (7 items)
* Demographics (10 items)

In [70]:
# premiere colonne : aime la musique ou non
music_ranking = data.ix[:,0:1]

# note pour chaque type de musique 
data_music = data.ix[:,0:19]
print(data_music.shape)

(1010, 19)


On construit la matrice de préférence. Pour cela, on classe sur chaque ligne les styles musicaux du mieux noté au moins bon. Ensuite on calcule les coefficients w(l1,l2) selon si l1 est préféré à l2 ou l'inverse. 
On calcule un coefficient final en moyennant w(l1,l2) sur l'ensemble des observations. Cela donnera la matrice de préférence sur l'ensemble des données 


In [77]:
# pour chaque ligne 
# pour chaque paire de colonne
# calculer omega(paire)

# sommer les paires, moyenner 
# matrice finale 

0       4
1       4
2       4
3       1
4       2
5       4
6       2
7       4
8       4
9       4
10      2
11      4
12      4
13      1
14      4
15      4
16      3
17      3
18      3
19      3
20      4
21      4
22     -1
23      4
24      4
25      0
26      3
27      3
28      3
29      3
       ..
980     3
981     3
982     3
983     2
984     3
985     4
986     3
987     4
988     4
989     4
990     3
991     4
992     3
993     3
994     4
995     3
996     3
997     3
998     4
999     3
1000    2
1001    2
1002    4
1003    4
1004    2
1005    3
1006    0
1007    3
1008    3
1009    0
Name: Rock, Length: 1010, dtype: int8


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().